In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Lib.module.risk_budgeting import *
import student_mixture as sm
from tqdm import tqdm
from Lib.module.movidas import *

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
np.random.seed(0)

In [10]:
freq = 'B'
df_all = pd.concat([pd.read_excel('Data/SP_RC.xlsx',index_col=0, header=[0,1]), pd.read_excel('Data/SP_RC2.xlsx',index_col=0, header=[0,1])])
df_all = df_all.dropna(how='all').dropna(axis=1)
df_all = df_all.pct_change().dropna()
df_all = df_all.replace([np.inf, -np.inf], np.nan)
df_all = df_all[~df_all.index.duplicated(keep='first')]
first_level_index = df_all.columns.get_level_values(0)


In [ ]:
gamma_sgd = {10:5, 25:1, 50:.5, 100:.25, 250:.1}
gamma_tamed = {10:1, 25:2.5, 50:5, 100:10, 250:25}

errors_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
errors_sgd_tamed = {10: [], 25: [], 50: [], 100: [], 250: []}
errors_smd = {10: [], 25: [], 50: [], 100: [], 250: []}

norms_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_sgd_tamed = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_smd = {10: [], 25: [], 50: [], 100: [], 250: []}

for i in tqdm(range(100)):
    for d in [10,25,50,100,250]:
        selected_indices = np.random.choice(range(len(first_level_index)), d, replace = False)
        assets = first_level_index[selected_indices]
        nb_asset = len(assets)
        df = df_all[assets]
        X = df.values

        ### Fit Student-t mixture (2 components) to chosen stock returns
        n_sm = 2
        SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [2.5,4]).fit(X)

        # ERC
        budgets = np.ones(nb_asset)/nb_asset 
        # Expected Shortfall alpha
        alpha = .95 

        SM_theta, optim_res = StudentMixtureExpectedShortfall(SM_model).solve_risk_budgeting(budgets, alpha, on_simplex=False, kappa=1, method=None, maxiter=15000)
        VaR_port = StudentMixtureExpectedShortfall(SM_model).value_at_risk(SM_theta, alpha)
        ES_port = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(SM_theta, alpha)
        risk_contribs = SM_theta * StudentMixtureExpectedShortfall(SM_model).expected_shortfall_gradient(SM_theta, alpha)

        optimal_loss = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(optim_res.x, alpha) - np.dot(budgets, np.log(optim_res.x))

        n_val=1000000
        X = SM_model.rvs(n_val)
        np.random.shuffle(X)

        gamma = gamma_tamed[d]
        gamma_sgd_ = gamma_sgd[d]

        y_sgd = budgets / np.std(X, axis=0)
        xi_sgd = 0
        c_sgd = .65

        y_sgd_tamed = budgets / np.std(X, axis=0)
        xi_sgd_tamed = 0
        c_sgd_tamed = .65

        y_smd = budgets / np.std(X, axis=0)
        xi_smd = 0
        c_smd = .65
        M = 100

        y_sgd_s = [y_sgd]
        xi_sgd_s = [xi_sgd]
        y_bar_sgd_s = [y_sgd]

        y_sgd_tamed_s = [y_sgd_tamed]
        xi_sgd_tamed_s = [xi_sgd_tamed]
        y_bar_sgd_tamed_s = [y_sgd_tamed]

        y_smd_s = [y_smd]
        xi_smd_s = [xi_smd]
        y_bar_smd_s = [y_smd]

        error_sgd = []
        error_sgd_tamed = []
        error_smd = []

        norm_sgd = []
        norm_sgd_tamed = []
        norm_smd = []

        freq_error = 100000

        for k in range(1, n_val):
            x = X[k]
            
            ### SGD
            # gradient
            step_size_sgd = gamma_sgd_/k**c_sgd
            indicator_sgd = -np.dot(y_sgd, x) - xi_sgd >= 0
            grad_y_sgd = -x/(1-alpha)*indicator_sgd - budgets/y_sgd
            grad_xi_sgd = 1 - (1 / (1 - alpha)) * indicator_sgd

            #descent
            y_sgd = y_sgd - step_size_sgd*grad_y_sgd
            y_sgd = np.where(y_sgd <= 0, 1e-04, y_sgd)
            xi_sgd = xi_sgd - step_size_sgd*grad_xi_sgd
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd

            ### SGD tamed
            # gradient
            step_size_sgd_tamed = gamma/k**c_sgd_tamed
            indicator_sgd_tamed = -np.dot(y_sgd_tamed, x) - xi_sgd_tamed >= 0
            grad_y_sgd_tamed = -x/(1-alpha)*indicator_sgd_tamed - budgets/y_sgd_tamed
            grad_xi_sgd_tamed = 1 - (1 / (1 - alpha)) * indicator_sgd_tamed

            #descent
            y_sgd_tamed_min = min(min(y_sgd_tamed),1)
            y_sgd_tamed = y_sgd_tamed - step_size_sgd*grad_y_sgd_tamed*y_sgd_tamed_min
            y_sgd_tamed = np.where(y_sgd_tamed <= 0, 1e-04, y_sgd_tamed)
            xi_sgd_tamed = xi_sgd_tamed - step_size_sgd_tamed*grad_xi_sgd_tamed
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd


            ### SMD
            # gradient
            step_size_smd = gamma/k**c_smd
            indicator_smd = -np.dot(y_smd, x) - xi_smd >= 0
            grad_y_smd = -x/(1-alpha)*indicator_smd - budgets/y_smd
            grad_xi_smd = 1 - (1 / (1 - alpha)) * indicator_smd

            y_smd_min = min(min(y_smd),1)
            y_smd = y_smd*np.exp(-step_size_smd*y_smd_min*grad_y_smd)
            xi_smd = xi_smd - step_size_smd*grad_xi_smd
            sum_y_smd = np.sum(y_smd)
            if sum_y_smd>M:
                y_smd = M/sum_y_smd*y_smd

            if k%freq_error==0:
                error_sgd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_sgd, alpha) - np.dot(budgets, np.log(y_sgd)) - optimal_loss)
                error_sgd_tamed.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_sgd_tamed, alpha) - np.dot(budgets, np.log(y_sgd_tamed)) - optimal_loss)
                error_smd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_smd, alpha) - np.dot(budgets, np.log(y_smd)) - optimal_loss)

                norm_sgd.append(np.mean(abs(SM_theta-y_sgd/y_sgd.sum())))
                norm_sgd_tamed.append(np.mean(abs(SM_theta-y_sgd_tamed/y_sgd_tamed.sum())))
                norm_smd.append(np.mean(abs(SM_theta-y_smd/y_smd.sum())))

        errors_sgd[d].append(error_sgd)
        errors_sgd_tamed[d].append(error_sgd_tamed)
        errors_smd[d].append(error_smd)

        norms_sgd[d].append(norm_sgd)
        norms_sgd_tamed[d].append(norm_sgd_tamed)
        norms_smd[d].append(norm_smd)

        np.save('Output/'+str(d)+'_sgd_errors_convergence.npy', np.array(errors_sgd[d]))
        np.save('Output/'+str(d)+'_smd_errors_convergence.npy', np.array(errors_smd[d]))     
        np.save('Output/'+str(d)+'_sgd_tamed_errors_convergence.npy', np.array(errors_sgd_tamed[d]))

        np.save('Output/'+str(d)+'_sgd_norms_convergence.npy', np.array(norms_sgd[d]))
        np.save('Output/'+str(d)+'_smd_norms_convergence.npy', np.array(norms_smd[d]))
        np.save('Output/'+str(d)+'_sgd_tamed_norms_convergence.npy', np.array(norms_sgd_tamed[d]))